In [3]:
from collections import defaultdict
from tabulate import tabulate

In [5]:
grammar = {'A': ['SB', 'B'],'S': ['a', 'Bc', 'ε'],'B': ['b', 'd']}

In [7]:
def first(symbol, computed_first={}):
    if symbol in computed_first:
        return computed_first[symbol]
    if not symbol.isupper(): 
        return {symbol}
    computed_first[symbol] = set()
    for production in grammar[symbol]:
        has_epsilon = True
        for char in production:
            if char in grammar:
                char_first = first(char, computed_first)
                computed_first[symbol].update(char_first - {'ε'})
                if 'ε' not in char_first:
                    has_epsilon = False
                    break
            else:
                computed_first[symbol].add(char)
                has_epsilon = False
                break
        if has_epsilon:
            computed_first[symbol].add('ε')
    return computed_first[symbol]

In [9]:
first_set = {nt: first(nt) for nt in grammar}
first_set

{'A': {'a', 'b', 'd'}, 'S': {'a', 'b', 'd', 'ε'}, 'B': {'b', 'd'}}

In [11]:
follow_set = defaultdict(set)
follow_set['A'].add('$') 

In [13]:
def follow(symbol):
    for lhs, productions in grammar.items():
        for production in productions:
            for i, char in enumerate(production):
                if char == symbol:
                    if i + 1 < len(production):
                        next_symbol = production[i + 1]
                        if next_symbol in grammar:
                            follow_set[char].update(first_set[next_symbol] - {'ε'})
                        else:
                            follow_set[char].add(next_symbol)
                    if i + 1 == len(production) or ('ε' in first_set.get(production[i + 1], {})):
                        follow_set[char].update(follow_set[lhs])

In [15]:
for _ in range(len(grammar)):
    for non_terminal in grammar:
        follow(non_terminal)

print("follow_set =", dict(follow_set))

follow_set = {'A': {'$'}, 'S': {'b', 'd'}, 'B': {'c', '$'}}


In [17]:
terminals = set()
for _, productions in grammar.items():
    for production in productions:
        for symbol in production:
            if not symbol.isupper() and symbol != 'ε':
                terminals.add(symbol)
terminals.add('$')

In [19]:
table = defaultdict(lambda: {t: "" for t in terminals})
for non_terminal, productions in grammar.items():
    for production in productions:
        first_of_production = first(production[0])
        for terminal in first_of_production - {'ε'}:
            if table[non_terminal][terminal]: 
                table[non_terminal][terminal] += f", {non_terminal} -> {production}"
            else:
                table[non_terminal][terminal] = f"{non_terminal} -> {production}"
        if 'ε' in first_of_production:
            for terminal in follow_set[non_terminal]:
                if table[non_terminal][terminal]:
                    table[non_terminal][terminal] += f", {non_terminal} -> {production}"
                else:
                    table[non_terminal][terminal] = f"{non_terminal} -> {production}"

In [21]:
table_data = [[nt] + [table[nt][t] for t in terminals] for nt in grammar]
headers = ["Terminal/NT"] + list(terminals)
print("LL(1) Parsing Table:")
print(tabulate(table_data, headers=headers, tablefmt="grid"))

LL(1) Parsing Table:
+---------------+---------+-----------------+---------+-----------------+-----+
| Terminal/NT   | $       | d               | a       | b               | c   |
+===============+=========+=================+=========+=================+=====+
| A             | A -> SB | A -> SB, A -> B | A -> SB | A -> SB, A -> B |     |
+---------------+---------+-----------------+---------+-----------------+-----+
| S             |         | S -> Bc, S -> ε | S -> a  | S -> Bc, S -> ε |     |
+---------------+---------+-----------------+---------+-----------------+-----+
| B             |         | B -> d          |         | B -> b          |     |
+---------------+---------+-----------------+---------+-----------------+-----+


In [23]:
def parse_string(input_string):
    stack = ['$', 'A'] 
    input_string += '$'  
    index = 0
    steps = []

    while stack:
        top = stack.pop()
        buffer = input_string[index:]
        if top == input_string[index]:
            steps.append(("".join(stack), buffer, f"Match {top}"))
            index += 1
        elif top in grammar:
            if table[top][input_string[index]]:
                productions = table[top][input_string[index]].split(', ')
                if len(productions) > 1:
                    return False, steps 
                production = productions[0]
                steps.append(("".join(stack), buffer, f"Apply {production}"))
                for symbol in reversed(production.split(' -> ')[1]):
                    if symbol != 'ε':
                        stack.append(symbol)
            else:
                return False, steps
        else:
            return False, steps
    return index == len(input_string), steps

In [25]:
# Valid String Case
input_string = "ab"
accepted, steps = parse_string(input_string)
print(f"String '{input_string}' is", "Accepted" if accepted else "Rejected")
print("\nParsing Steps:")
headers = ["Stack", "Buffer", "Action"]
print(tabulate(steps, headers=headers, tablefmt="grid"))

String 'ab' is Accepted

Parsing Steps:
+---------+----------+---------------+
| Stack   | Buffer   | Action        |
+=========+==========+===============+
| $       | ab$      | Apply A -> SB |
+---------+----------+---------------+
| $B      | ab$      | Apply S -> a  |
+---------+----------+---------------+
| $B      | ab$      | Match a       |
+---------+----------+---------------+
| $       | b$       | Apply B -> b  |
+---------+----------+---------------+
| $       | b$       | Match b       |
+---------+----------+---------------+
|         | $        | Match $       |
+---------+----------+---------------+


In [27]:
# Invalid String Case
input_string = "abd"
accepted, steps = parse_string(input_string)
print(f"String '{input_string}' is", "Accepted" if accepted else "Rejected")
print("\nParsing Steps:")
headers = ["Stack", "Buffer", "Action"]
print(tabulate(steps, headers=headers, tablefmt="grid"))

String 'abd' is Rejected

Parsing Steps:
+---------+----------+---------------+
| Stack   | Buffer   | Action        |
+=========+==========+===============+
| $       | abd$     | Apply A -> SB |
+---------+----------+---------------+
| $B      | abd$     | Apply S -> a  |
+---------+----------+---------------+
| $B      | abd$     | Match a       |
+---------+----------+---------------+
| $       | bd$      | Apply B -> b  |
+---------+----------+---------------+
| $       | bd$      | Match b       |
+---------+----------+---------------+
